## Домашка (10 баллов)


Обучите модель с минимум 15 слоями, где у каждого слоя разные параметры (Dropout, Conv1d и Pooling, Dense считаются слоями, остальное нет). Как мимнимум 4 слоя должны быть наложены друг на друга и как минимум 2 параллельных слоя (последовательности слоев). Должен быть хотя бы один слой каждого типа.

При обучении используйте колбек для отслеживания лучшей модели. Ориентируйтесь на ф1 меру. Качество модели не должно быть околонулевым. Если метрики не растут, то попробуйте пообучать подольше или перестроить саму сеть.

Советы: Начните с небольших сетей и постепенно добавляйте, не пытайтесь сразу собрать все слои. Иногда кернел может крашиться просто так или из-за слишком больших матриц.


In [ ]:
%pip install --upgrade keras
%pip install torch torchvision torchaudio
%pip install tensorflow

In [2]:
import os
os.environ["KERAS_BACKEND"] = "torch"

import tensorflow
import keras

In [3]:
import pandas as pd
import numpy as np
from string import punctuation
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
data = pd.read_csv('lenta_40k.csv.zip')
data

,text,topic
0,Россия должна сотрудничать с Всемирным антидоп...,Спорт
1,Уголовный суд Кувейта 28 июня освободил под за...,Мир
2,Французский журнал Charlie Hebdo опубликовал н...,Интернет и СМИ
3,В Петербурге в доме № 53 по улице Лени Голиков...,Россия
4,"В московском аэропорту ""Домодедово"" задержан г...",Россия
...,...,...
44351,Российский Минфин может пересмотреть минимальн...,Экономика
44352,Сотрудники Службы безопасности Украины (СБУ) с...,Бывший СССР
44353,Америке следует задуматься над поставками на У...,Силовые структуры
44354,"Опознаны тела 71 человека, погибшего в результ...",Мир


In [5]:
def preprocess(text):
    tokens = text.lower().split()
    tokens = [token.strip(punctuation) for token in tokens]
    return tokens

In [6]:
vocab = Counter()

for text in data.text:
    vocab.update(preprocess(text))

filtered_vocab = set()

for word in vocab:
    if vocab[word] > 30:
        filtered_vocab.add(word)

word2id = {'PAD':0}

for word in filtered_vocab:
    word2id[word] = len(word2id)

id2word = {i:word for word, i in word2id.items()}

In [7]:
X = []

for text in data.text:
    tokens = preprocess(text)
    ids = [word2id.get(token, 1) for token in tokens]
    X.append(ids)

MAX_LEN = max(len(x) for x in X)
MEAN_LEN = np.median([len(x) for x in X])
MAX_LEN = int(MEAN_LEN + 30)

In [8]:
X = keras.preprocessing.sequence.pad_sequences(X, maxlen=MAX_LEN)

In [9]:
id2label = {i:label for i, label in enumerate(set(data.topic.values))}
label2id = {l:i for i, l in id2label.items()}

In [10]:
y = keras.utils.to_categorical([label2id[label] for label in data.topic.values])

In [11]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.05)

In [12]:
inputs = keras.layers.Input(shape=(MAX_LEN,))
embeddings = keras.layers.Embedding(input_dim=len(word2id), output_dim=30)(inputs, )
mean = keras.layers.Flatten()(embeddings)
dense = keras.layers.Dense(30, activation='relu')(mean)
outputs = keras.layers.Dense(len(label2id), activation='softmax')(dense)

model = keras.Model(inputs=inputs, outputs=outputs)
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[keras.metrics.F1Score(name='f1score')])

In [13]:
model.fit(X_train, y_train,
          validation_data=(X_valid, y_valid),
          batch_size=2000,
          epochs=25)

Epoch 1/25
22/22 [==============================] - 6s 208ms/step - loss: 2.7386 - f1score: 0.0407 - val_loss: 2.4799 - val_f1score: 0.0187
Epoch 2/25
22/22 [==============================] - 4s 165ms/step - loss: 2.3219 - f1score: 0.0304 - val_loss: 2.1702 - val_f1score: 0.0431
Epoch 3/25
22/22 [==============================] - 3s 156ms/step - loss: 1.9802 - f1score: 0.0749 - val_loss: 1.8502 - val_f1score: 0.1188
Epoch 4/25
22/22 [==============================] - 4s 160ms/step - loss: 1.6171 - f1score: 0.1871 - val_loss: 1.4789 - val_f1score: 0.2136
Epoch 5/25
22/22 [==============================] - 4s 197ms/step - loss: 1.2573 - f1score: 0.2490 - val_loss: 1.2430 - val_f1score: 0.2519
Epoch 6/25
22/22 [==============================] - 3s 155ms/step - loss: 1.0171 - f1score: 0.3275 - val_loss: 1.0746 - val_f1score: 0.3354
Epoch 7/25
22/22 [==============================] - 3s 156ms/step - loss: 0.8239 - f1score: 0.4130 - val_loss: 0.9529 - val_f1score: 0.4025
Epoch 8/25
22/22 [==

Модель, обученная на двух слоях Dense, и трёх неучитываемых.

In [14]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200)]             0         
                                                                 
 embedding (Embedding)       (None, 200, 30)           722760    
                                                                 
 flatten (Flatten)           (None, 6000)              0         
                                                                 
 dense (Dense)               (None, 30)                180030    
                                                                 
 dense_1 (Dense)             (None, 19)                589       
                                                                 
Total params: 903379 (3.45 MB)
Trainable params: 903379 (3.45 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [15]:
inputs = keras.layers.Input(shape=(MAX_LEN,))
embeddings = keras.layers.Embedding(input_dim=len(word2id), output_dim=30)(inputs, )
conv1 = keras.layers.Conv1D(kernel_size=5, filters=32, strides=1)(embeddings)
conv2 = keras.layers.Conv1D(kernel_size=5, filters=32, strides=1, activation='relu')(conv1)
concat = keras.layers.Flatten()(conv2)
dense = keras.layers.Dense(30, activation='relu')(concat)
outputs = keras.layers.Dense(len(label2id), activation='softmax')(dense)

model = keras.Model(inputs=inputs, outputs=outputs)
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[keras.metrics.F1Score(name='f1score')])

In [16]:
model.fit(X_train, y_train,
          validation_data=(X_valid, y_valid),
          batch_size=2000,
          epochs=25)

Epoch 1/25
22/22 [==============================] - 23s 1s/step - loss: 2.6063 - f1score: 0.0278 - val_loss: 2.4168 - val_f1score: 0.0183
Epoch 2/25
22/22 [==============================] - 24s 1s/step - loss: 2.2912 - f1score: 0.0317 - val_loss: 2.1222 - val_f1score: 0.0423
Epoch 3/25
22/22 [==============================] - 24s 1s/step - loss: 1.9061 - f1score: 0.1030 - val_loss: 1.6650 - val_f1score: 0.1396
Epoch 4/25
22/22 [==============================] - 23s 1s/step - loss: 1.4519 - f1score: 0.1980 - val_loss: 1.3261 - val_f1score: 0.2408
Epoch 5/25
22/22 [==============================] - 24s 1s/step - loss: 1.0732 - f1score: 0.2844 - val_loss: 1.1472 - val_f1score: 0.2981
Epoch 6/25
22/22 [==============================] - 23s 1s/step - loss: 0.8330 - f1score: 0.3503 - val_loss: 1.1227 - val_f1score: 0.3218
Epoch 7/25
22/22 [==============================] - 23s 1s/step - loss: 0.6725 - f1score: 0.4029 - val_loss: 1.1429 - val_f1score: 0.3576
Epoch 8/25
22/22 [================

Модель, обученнная на четырёх слоях, Dense и ConvID.

In [17]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 200)]             0         
                                                                 
 embedding_1 (Embedding)     (None, 200, 30)           722760    
                                                                 
 conv1d (Conv1D)             (None, 196, 32)           4832      
                                                                 
 conv1d_1 (Conv1D)           (None, 192, 32)           5152      
                                                                 
 flatten_1 (Flatten)         (None, 6144)              0         
                                                                 
 dense_2 (Dense)             (None, 30)                184350    
                                                                 
 dense_3 (Dense)             (None, 19)                589 

In [18]:
inputs = keras.layers.Input(shape=(MAX_LEN,))
embeddings = keras.layers.Embedding(input_dim=len(word2id), output_dim=30)(inputs, )
drop1 = keras.layers.Dropout(0.1)(embeddings)

conv1 = keras.layers.Conv1D(kernel_size=5, filters=32, strides=1)(embeddings)
conv2 = keras.layers.Conv1D(kernel_size=5, filters=32, strides=1, activation='relu')(conv1)

drop2 = keras.layers.Dropout(0.1)(conv2)

concat = keras.layers.Flatten()(drop2)
dense = keras.layers.Dense(30, activation='relu')(concat)
outputs = keras.layers.Dense(len(label2id), activation='softmax')(dense)

model = keras.Model(inputs=inputs, outputs=outputs)
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[keras.metrics.F1Score(name='f1score')])

In [19]:
model.fit(X_train, y_train,
          validation_data=(X_valid, y_valid),
          batch_size=2000,
          epochs=25)

Epoch 1/25
22/22 [==============================] - 28s 1s/step - loss: 2.5896 - f1score: 0.0211 - val_loss: 2.4030 - val_f1score: 0.0182
Epoch 2/25
22/22 [==============================] - 26s 1s/step - loss: 2.2872 - f1score: 0.0234 - val_loss: 2.1236 - val_f1score: 0.0482
Epoch 3/25
22/22 [==============================] - 26s 1s/step - loss: 1.9261 - f1score: 0.0865 - val_loss: 1.7640 - val_f1score: 0.1109
Epoch 4/25
22/22 [==============================] - 26s 1s/step - loss: 1.5483 - f1score: 0.1549 - val_loss: 1.4276 - val_f1score: 0.2048
Epoch 5/25
22/22 [==============================] - 26s 1s/step - loss: 1.1511 - f1score: 0.2645 - val_loss: 1.2092 - val_f1score: 0.2713
Epoch 6/25
22/22 [==============================] - 26s 1s/step - loss: 0.9166 - f1score: 0.3308 - val_loss: 1.1839 - val_f1score: 0.3114
Epoch 7/25
22/22 [==============================] - 26s 1s/step - loss: 0.7546 - f1score: 0.3908 - val_loss: 1.2170 - val_f1score: 0.3180
Epoch 8/25
22/22 [================

Модель, обученная на пяти слоях. Дальше добавим бОльшее количество слоёв с помощью цикла.

In [20]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 200)]             0         
                                                                 
 embedding_2 (Embedding)     (None, 200, 30)           722760    
                                                                 
 conv1d_2 (Conv1D)           (None, 196, 32)           4832      
                                                                 
 conv1d_3 (Conv1D)           (None, 192, 32)           5152      
                                                                 
 dropout_1 (Dropout)         (None, 192, 32)           0         
                                                                 
 flatten_2 (Flatten)         (None, 6144)              0         
                                                                 
 dense_4 (Dense)             (None, 30)                1843

In [21]:
inputs = keras.layers.Input(shape=(MAX_LEN,))
embeddings = keras.layers.Embedding(input_dim=len(word2id), output_dim=30)(inputs, )
drop1 = keras.layers.Dropout(0.1)(embeddings)

convs = []

for ks in [2,4,6,8,10]:
    conv1 = keras.layers.Conv1D(kernel_size=ks, filters=32, padding='same',strides=1)(embeddings)
    drop2 = keras.layers.Dropout(0.1)(conv1)
    conv2 = keras.layers.Conv1D(kernel_size=ks, filters=32, padding='same',strides=1,
                                  kernel_regularizer='l2', activation='relu')(drop2)
    pool = keras.layers.AveragePooling1D(pool_size=5)(conv2)
    conv3 = keras.layers.Conv1D(kernel_size=ks, filters=32, padding='same',strides=2,
                                  kernel_regularizer='l2', activation='relu')(pool)
    convs.append(conv3)


concat = keras.layers.concatenate(convs, axis=2)

conv_global = keras.layers.Conv1D(kernel_size=5, filters=32, strides=1)(concat)
flatten = keras.layers.Flatten()(conv_global)
dense = keras.layers.Dense(50, activation='relu')(flatten)
outputs = keras.layers.Dense(len(label2id), activation='softmax')(dense)

model = keras.Model(inputs=inputs, outputs=outputs)
optimizer = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=[keras.metrics.F1Score(name='f1score')])

In [32]:
checkpoint = keras.callbacks.ModelCheckpoint('model.weights.keras_checkpoint', # названия файла
                                                monitor='val_f1score', # за какой метрикой следить
                                                verbose=1, # будет печатать что происходит
                                                save_weights_only=True, # если нужно только веса сохранить
                                                save_best_only=True, # сохранять только лучшие
                                                mode='max', # если метрика должна расти, то тут max и min если наоборот
                                                save_freq='epoch' # как часто вызывать
                                               )

early_stop = keras.callbacks.EarlyStopping(monitor='val_f1score',
                                              min_delta=0.01, # какая разница считается как улучшение
                                              patience=3, # сколько эпох терпеть отсутствие улучшений
                                              verbose=1,
                                              mode='max',
                                              )

In [34]:
model.fit(X_train, y_train,
          validation_data=(X_valid, y_valid),
          batch_size=2000,
          epochs=25)
          #callbacks=[checkpoint, early_stop])

Epoch 1/25
22/22 [==============================] - 139s 6s/step - loss: 0.6508 - f1score: 0.4826 - val_loss: 1.9340 - val_f1score: 0.3014
Epoch 2/25
22/22 [==============================] - 138s 6s/step - loss: 0.6339 - f1score: 0.4910 - val_loss: 1.8809 - val_f1score: 0.3179
Epoch 3/25
22/22 [==============================] - 136s 6s/step - loss: 0.5684 - f1score: 0.5175 - val_loss: 2.0398 - val_f1score: 0.3169
Epoch 4/25
22/22 [==============================] - 140s 6s/step - loss: 0.5140 - f1score: 0.5368 - val_loss: 2.1640 - val_f1score: 0.3241
Epoch 5/25
22/22 [==============================] - 143s 6s/step - loss: 0.4704 - f1score: 0.5566 - val_loss: 2.3535 - val_f1score: 0.3096
Epoch 6/25
22/22 [==============================] - 138s 6s/step - loss: 0.4391 - f1score: 0.5724 - val_loss: 2.4375 - val_f1score: 0.3215
Epoch 7/25
22/22 [==============================] - 137s 6s/step - loss: 0.4053 - f1score: 0.5831 - val_loss: 2.4987 - val_f1score: 0.3387
Epoch 8/25
22/22 [=========

In [35]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 200)]                0         []                            
                                                                                                  
 embedding_3 (Embedding)     (None, 200, 30)              722760    ['input_4[0][0]']             
                                                                                                  
 conv1d_4 (Conv1D)           (None, 200, 32)              2912      ['embedding_3[0][0]']         
                                                                                                  
 conv1d_7 (Conv1D)           (None, 200, 32)              3872      ['embedding_3[0][0]']         
                                                                                            